In [1]:
import numpy as np
import timor
import argparse
import pygad
import os
from util import *
from timor.Module import *
from timor.utilities.visualization import animation
# import matplotlib.pyplot as plt
# import itertools
# from mpl_toolkits.mplot3d import Axes3D
from tqdm import tqdm
from timor import ModuleAssembly, ModulesDB
from timor.Bodies import Body, Connector, Gender
from timor.configuration_search.GA import GA
from timor.utilities.visualization import MeshcatVisualizer, clear_visualizer
from timor.utilities.dtypes import Lexicographic
from timor.utilities.transformation import Transformation
from timor.utilities.spatial import rotX, rotY, rotZ
from timor.Module import AtomicModule, ModulesDB, ModuleHeader
from timor.Joints import Joint
from timor.Geometry import Box, ComposedGeometry, Cylinder, Sphere, Mesh

from reachability import Reachability
from generate_module import create_i_links, create_eef, create_revolute_joint, base

2025-02-24 14:54:28,774 Timor INFO Loading custom configurations from /home/mscsim/.config/timor.config
2025-02-24 14:54:29,302 Timor INFO Getting robot modrob-gen2.
2025-02-24 14:54:29,303 Timor WARNING Could not fetch robot modrob-gen2 due to [Errno 17] File exists: '/home/mscsim/anaconda3/envs/timor-env/lib/python3.10/site-packages/cache/robots/modrob-gen2'.


In [2]:
how_many_times_to_split_angle_range = 30
world_resolution = 0.01
world_dimension = [1.00, 1.00, 1.00]
num_threads = 5
our_hyperparameters = {
    'population_size': 10,
    'num_generations': 50,
    'num_genes': 6,
    'save_solutions_dir': None
}

In [2]:
#r_4310_base = create_revolute_joint("assets/Assem_4310_BASE/Assem_4310_BASE/urdf/Assem_4310_BASE.urdf")
r_4310_base = base("assets/Assem_4310_BASE/Assem_4310_BASE/urdf/Assem_4310_BASE.urdf")
r_4305_joint = create_revolute_joint("assets/Assem_4305_JOINT/Assem_4305_JOINT/urdf/Assem_4305_JOINT.urdf")
r_4310_joint = create_revolute_joint("assets/Assem_4310_JOINT/Assem_4310_JOINT/urdf/Assem_4310_JOINT.urdf")

# Links

baseto4310_links = create_i_links(rod_name="baseto4310")
r4310to4305_links = create_i_links(rod_name="r4310to4305")
r4310to4310_links = create_i_links(rod_name="r4310to4310")

# eef = create_eef()

# Create database
db = ModulesDB()
db.add(r_4310_base)
db.add(r_4310_joint)
db.add(r_4305_joint)
db = db.union(baseto4310_links)
db = db.union(r4310to4305_links)
db = db.union(r4310to4310_links)

# db = db.union(eef)
viz = db.debug_visualization()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [12]:
modules = ('base_rev_joint','baseto4310-0.3','motor4310_rev_joint', 'r4310to4310-0.3', 'motor4310_rev_joint', 'r4310to4305-0.15', 'motor4305_rev_joint')
B = ModuleAssembly.from_serial_modules(db, modules)
#print(B.nJoints)
robot = B.to_pin_robot()
print(robot.has_self_collision())
viz = robot.visualize()

False
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7007/static/


In [15]:
from timor.utilities.visualization import animation
q0 = np.zeros((robot.dof,))
print(robot.dof)
q1 = robot.random_configuration()
q2 = np.array([np.pi, -np.pi/2, np.pi/2, np.pi/2])
print(q1)
robot.update_configuration(q1)
print(robot.has_self_collision())
viz = robot.visualize()
#print(q1)

# trajectory = np.linspace(q0, q1)
# print(trajectory)
# animation(robot, trajectory, dt=.1)

4
[ 1.56873826e-03 -2.27632885e+00 -6.89821853e-01 -1.94983411e-01]
False
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7010/static/


In [16]:
from reachability import Reachability

how_many_times_to_split_angle_range = 30
world_resolution = 0.01
world_dimension = [1.00, 1.00, 1.00]
num_threads = 5

reach = Reachability(robot)
reach.reachability_random_sample(num_samples = 100000)
valid_poses = reach.reachability_random_sample(num_samples = 100000)
print("Valid poses:" , valid_poses)
reachable_space = reach.find_reachibility_percentage(world_dim=world_dimension, world_res=world_resolution)


  0%|          | 0/100000 [00:00<?, ?it/s]/home/mscsim/anaconda3/envs/timor-env/lib/python3.10/site-packages/timor/Robot.py:828: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(np.linalg.det(J @ J.T))
100%|██████████| 100000/100000 [00:12<00:00, 8041.35it/s]

Valid poses: [((-0.11, 0.27, 0.18), nan), ((-0.08, 0.13, 0.8), nan), ((-0.12, -0.21, 0.72), nan), ((-0.12, -0.28, 0.67), 0.0), ((0.18, 0.34, 0.55), 0.0), ((-0.03, 0.07, 0.78), nan), ((0.23, 0.29, 0.39), 1.6112552650598623e-18), ((-0.35, -0.06, 0.6), nan), ((-0.05, 0.33, 0.6), 3.4643463712241254e-18), ((-0.01, 0.35, 0.27), 1.4847087436206217e-18), ((-0.19, -0.02, 0.74), nan), ((0.21, 0.27, 0.55), 0.0), ((-0.19, 0.26, 0.6), nan), ((0.12, 0.04, 0.81), 0.0), ((0.03, 0.13, 0.81), 1.1087550051105534e-18), ((0.03, 0.1, 0.8), nan), ((0.18, 0.34, 0.42), nan), ((-0.02, -0.17, 0.05), 0.0), ((0.32, 0.26, 0.38), nan), ((0.1, 0.27, 0.18), nan), ((-0.21, -0.32, 0.27), 3.5985310970757786e-18), ((-0.26, -0.09, 0.71), 0.0), ((-0.18, -0.17, 0.69), 0.0), ((0.08, -0.39, 0.35), 2.6847755194334218e-18), ((0.28, 0.23, 0.6), nan), ((-0.22, 0.22, 0.2), nan), ((0.31, 0.08, 0.18), nan), ((-0.05, -0.32, 0.19), 8.98169389073827e-19), ((0.37, 0.15, 0.45), 1.7366841427117993e-18), ((-0.03, 0.07, 0.78), nan), ((0.16, 

In [19]:
print(valid_poses[0])
print(len(valid_poses))

((-0.11, 0.27, 0.18), nan)
87451
